In [ ]:
import torch
import transformers
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments, LlamaForCausalLM

In [ ]:
from huggingface_hub import login

MY_HF_TOKEN = "input your hf token"
model_id = "meta-llama/Llama-3.2-3B-Instruct"

login("MY_HF_TOKEN")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# 모델 및 토크나이저 불러오기

model = LlamaForCausalLM.from_pretrained(model_id, use_auth_token=MY_HF_TOKEN, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=MY_HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3220: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
# 학습시킬 데이터 csv 파일 불러와서 각 질병마다 하나의 텍스트로 정리
disease_path ='/content/drive/MyDrive/제4회 AI교육 해커톤/disease_train.csv'
disease_path = 'input disease path'
data = pd.read_csv(disease_path)

data['text'] = data.apply(
    lambda row: f"질병: {row['질병']}\n설명: {row['설명']}\n원인: {row['원인']}\n증상: {row['증상']}\n치료방법: {row['치료방법']}",
    axis=1
)

In [ ]:
# 데이터를 학습시키기 위해 토크나이저 정의

inputs = tokenizer(
    list(data['text']),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

In [ ]:
# 학습시킬 데이터 텍스트를 인코딩하기 위한 클래스 정의

class DiseaseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
# 학습시킬 데이터 텍스트를 인코딩

dataset = DiseaseDataset(inputs)

In [ ]:
# TrainingArguments 설정

training_args = TrainingArguments(
    output_dir="./results",          # 결과 저장 경로
    num_train_epochs=3,              # 에폭 수
    per_device_train_batch_size=2,   # 배치 크기
    save_steps=10_000,               # 저장 주기
    save_total_limit=2,              # 최대 저장 파일 수
    logging_dir="./logs",            # 로깅 경로
)

In [ ]:
# 모델 학습

trainer = Trainer(
    model=model,                     # 학습할 모델
    args=training_args,              # 학습 인자
    train_dataset=dataset             # 학습 데이터셋
)

trainer.train()

In [ ]:
# 예측된 질병에 대한 설명을 제시하도록 프롬프트 설정 후 생성된 설명을 반환하는 함수 정의

def generate_disease_description(disease_name):
    prompt = f"질병: {disease_name}\n이 질병에 대해 설명해 주세요."
    response = pipeline(prompt, max_length=300, num_return_sequences=1, temperature=0.3)
    return response[0]['generated_text']

In [ ]:
# 예시로 '성충_응애'로 예측했을 때 생성된 설명을 출력

disease_name = "성충_응애"
description = generate_disease_description(disease_name)
print(description)